In [1]:
# 1) OpenAI setup (API key uit .env)

!pip install -q openai python-dotenv

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("API key not found. Zet OPENAI_API_KEY in je .env")

client = OpenAI(api_key=OPENAI_API_KEY)

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install -q pandas openpyxl

import os
import time
import warnings
import pandas as pd

os.environ["LC_ALL"] = "C.UTF-8"
os.environ["LANG"] = "C.UTF-8"

warnings.filterwarnings(
    "ignore",
    message="Unknown extension is not supported and will be removed",
    category=UserWarning,
    module="openpyxl"
)

# ====== CONFIG ======
EXCEL_PATH = r"/home/yasin/Git/Cross-domain-recommender/experiment/notebook/complete dataset.xlsx"
SHEET_NAME = 0

NAME_COL = "name"
OVERVIEW_COL = "source_overview"
TYPE_COL = "item_type"
GENRE_COL = "Simplified genre"
ENRICHED_COL = "enriched_description"

# Optionele kolommen (als ze bestaan worden ze gebruikt, anders leeg)
YEAR_COL = "Year"
SOURCE_GENRES_COL = "source_genres"
CREATED_BY_COL = "created_by"

MODEL = "gpt-4o-mini"
MAX_RETRIES = 5
SLEEP_BETWEEN_CALLS_SEC = 0.3


def is_missing(value) -> bool:
    if value is None:
        return True
    try:
        return pd.isna(value)
    except Exception:
        return str(value).strip() == ""


def safe_get_str(df: pd.DataFrame, row_idx: int, col: str) -> str:
    """Return string value for cell, or empty string if column missing/NaN/blank."""
    if col not in df.columns:
        return ""
    v = df.at[row_idx, col]
    if is_missing(v):
        return ""
    return str(v).strip()


def build_prompt_v2(
    name: str,
    item_type: str,
    year: str,
    source_genres: str,
    simplified_genre: str,
    created_by: str,
    overview: str
) -> str:
    # Fallbacks zodat placeholders altijd bestaan
    name = (name or "").strip()
    item_type = (item_type or "").strip()
    year = (year or "").strip()
    source_genres = (source_genres or "").strip()
    simplified_genre = (simplified_genre or "").strip()
    created_by = (created_by or "").strip()
    overview = (overview or "").strip()

    # Maak de INPUT-sectie altijd geldig, ook als velden leeg zijn
    prompt = f"""
ROLE
You are an expert Data Curator. You receive raw data about a {item_type if item_type else "media item"}
and produce one coherent, semantically rich, atmospheric synopsis.

INPUT
---
Title: {name}
Year: {year}
Genre Tags: {source_genres} (Primary Genre: {simplified_genre})
Creators/Authors: {created_by}
Raw Description: {overview if overview else "N/A"}
---

INSTRUCTIONS
• Write a spoiler-free synopsis of 200–300 words.
• Focus on setting, central tensions, themes, and atmosphere.
• Rewrite inconsistent, minimal, or marketing-like source material into a smooth,
  neutral-literary narrative style.
• Use your own knowledge of the item only to deepen atmosphere, thematic resonance,
  and implicit tension — never to introduce factual details not inferable from the input.
• Do not invent plot points or specify events that are not supported by the input; interpret, do not fabricate.

• Subtly but clearly reflect the provided genre tags in mood, thematic accents,
  world-building, and terminology. Allow elements of {simplified_genre} and the broader tags
  to resonate through the synopsis without naming genres explicitly.
• Integrate 2–4 genre-typical features (e.g., Sci-Fi elements, adventure-driven dynamics,
  emotional or moral conflict) as semantic anchors for embedding quality.

• Avoid all metadata: no years, no factual trivia, no names of actors, directors, or authors.
• No meta-language: do not say things like “this film/series/book tells the story…”
• Maintain stylistic consistency: neutral-literary tone with clear structure:
  setting introduction → core conflict → thematic layer → closing atmosphere.

OUTPUT
Only output the final synopsis.
""".strip()

    return prompt


def generate_enriched_description(name, item_type, year, source_genres, simplified_genre, created_by, overview) -> str:
    prompt = build_prompt_v2(
        name=name,
        item_type=item_type,
        year=year,
        source_genres=source_genres,
        simplified_genre=simplified_genre,
        created_by=created_by,
        overview=overview,
    )

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "Only output the final synopsis. No extra text."},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.4,
            )

            text = (response.choices[0].message.content or "").strip()
            if len(text) < 120:
                raise ValueError("Output too short")
            return text

        except Exception as exc:
            wait = min(2 ** attempt, 20)
            print(f"[WARN] Attempt {attempt}/{MAX_RETRIES} failed: {exc}")
            time.sleep(wait)

    raise RuntimeError("AI call failed after all retries")


# ====== READ EXCEL ======
raw = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)
df = raw[next(iter(raw))] if isinstance(raw, dict) else raw

missing_columns = [c for c in (NAME_COL, ENRICHED_COL) if c not in df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}")

# Altijd verrijken ook al als enriched_description bestaat, zodat we hetzelfde prompt gebruiken, om te verrijken.
rows_to_process = df.index.tolist()
print(f"Rows to enrich: {len(rows_to_process)} / {len(df)}")

for index, row_idx in enumerate(rows_to_process, start=1):
    name = safe_get_str(df, row_idx, NAME_COL)
    if not name:
        continue

    item_type = safe_get_str(df, row_idx, TYPE_COL)
    simplified_genre = safe_get_str(df, row_idx, GENRE_COL)
    overview = safe_get_str(df, row_idx, OVERVIEW_COL)

    year = safe_get_str(df, row_idx, YEAR_COL)
    source_genres = safe_get_str(df, row_idx, SOURCE_GENRES_COL)
    created_by = safe_get_str(df, row_idx, CREATED_BY_COL)

    print(f"[{index}/{len(rows_to_process)}] {name}")
    df.at[row_idx, ENRICHED_COL] = generate_enriched_description(
        name=name,
        item_type=item_type,
        year=year,
        source_genres=source_genres,
        simplified_genre=simplified_genre,
        created_by=created_by,
        overview=overview,
    )

    time.sleep(SLEEP_BETWEEN_CALLS_SEC)

# ====== WRITE OUTPUT ======
output_path = os.path.join(os.getcwd(), "..", "..", "data", "complete dataset.xlsx")

# Zorg dat de map bestaat
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_excel(output_path, index=False)
print("Saved:", output_path)

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Rows to enrich: 105 / 105
[1/105] The Mandalorian
[2/105] The 100
[3/105] Westworld
[4/105] Black Mirror
[5/105] The X-Files
[6/105] Game of Thrones
[7/105] Wednesday
[8/105] Attack on Titan
[9/105] House of the Dragon
[10/105] The Originals
[11/105] Money Heist
[12/105] Sherlock
[13/105] Better Call Saul
[14/105] Mr. Robot
[15/105] Dexter
[16/105] Breaking Bad
[17/105] The Good Doctor
[18/105] Grey's Anatomy
[19/105] Euphoria
[20/105] Peaky Blinders
[21/105] The Haunting of Hill House
[22/105] Scream Queens
[23/105] 1899
[24/105] The Outsider
[25/105] It
[26/105] The Big Bang Theory
[27/105] Friends
[28/105] How I Met Your Mother
[29/105] The Office
[30/105] Brooklyn Nine-Nine
[31/105] Yo soy Betty, la fea
[32/105] Teresa
[33/105] La hija del Mariachi
[34/105] Amor Real
[35/105] My Heart is Yours
[36/10